In [1]:
import os
import re

import jax
import tensorflow as tf

import ml_collections

from molnet.data import input_pipeline
#from configs import root_dirs, default

2024-10-31 17:06:40.914203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730387200.926416 2543999 cuda_dnn.cc:8498] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730387200.930201 2543999 cuda_blas.cc:1410] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
config = ml_collections.ConfigDict()
config.root_dir = '/u/79/kurkil1/unix/work/molnet/puhtidata'

config.train_molecules = (0, 1000)
config.val_molecules = (1000, 2000)
config.test_molecules = (2000, 3000)

In [3]:
rng = jax.random.PRNGKey(0)
datarng, rng = jax.random.split(rng)
datasets = input_pipeline.get_datasets(rng, config)

I0000 00:00:1730387208.823523 2543999 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1452 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:01:00.0                                                   , compute capability: 8.6


TypeError: in user code:

    File "/m/home/home7/79/kurkil1/unix/git/molecule-net/molnet/data/input_pipeline.py", line 53, in None  *
        lambda path: tf.data.Dataset.load(path, element_spec=element_spec)

    TypeError: TF_DATA_SnapshotMetadataFilePath(): incompatible function arguments. The following argument types are supported:
        1. (arg0: str) -> str
    
    Invoked with: <tf.Tensor 'args_0:0' shape=() dtype=string>


In [ ]:
filenames = sorted(os.listdir(config.root_dir))
filenames = [
    os.path.join(config.root_dir, f)
    for f in filenames
    if f.startswith("maps_")
]

if len(filenames) == 0:
    raise ValueError(f"No files found in {config.root_dir}.")

# Partition the filenames into train, val, and test.
def filter_by_molecule_number(
    filenames, start, end
):
    def filter_file(filename: str, start: int, end: int) -> bool:
        filename = os.path.basename(filename)
        file_start, file_end = [int(val) for val in re.findall(r"\d+", filename)]
        return start <= file_start and file_end <= end

    return [f for f in filenames if filter_file(f, start, end)]


In [ ]:
files_by_split = {
    "train": filter_by_molecule_number(filenames, *config.train_molecules),
    "val": filter_by_molecule_number(filenames, *config.val_molecules),
    "test": filter_by_molecule_number(filenames, *config.test_molecules),
}

print(files_by_split)

{'train': ['/u/79/kurkil1/unix/work/molnet/puhtidata/maps_000000_001000'], 'val': ['/u/79/kurkil1/unix/work/molnet/puhtidata/maps_001000_002000'], 'test': ['/u/79/kurkil1/unix/work/molnet/puhtidata/maps_002000_003000']}


In [ ]:
element_spec = tf.data.Dataset.load(filenames[0]).element_spec
print(element_spec)

train_files = files_by_split["train"]
dataset = tf.data.Dataset.from_tensor_slices(train_files)

print(dataset)


{'images': TensorSpec(shape=(128, 128, 10), dtype=tf.float16, name=None), 'xyz': TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), 'atom_map': TensorSpec(shape=(5, 128, 128, 21), dtype=tf.float16, name=None)}
<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


I0000 00:00:1730386992.683647 2543021 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9247 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:01:00.0                                                   , compute capability: 8.6


In [ ]:
dataset = dataset.interleave(
    lambda path: tf.data.Dataset.load(path, element_spec=element_spec),
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=True,
)


TypeError: in user code:

    File "/tmp/ipykernel_2543021/2036790377.py", line 2, in None  *
        lambda path: tf.data.Dataset.load(path, element_spec=element_spec)

    TypeError: TF_DATA_SnapshotMetadataFilePath(): incompatible function arguments. The following argument types are supported:
        1. (arg0: str) -> str
    
    Invoked with: <tf.Tensor 'args_0:0' shape=() dtype=string>
